# Tutorial de PyTorch 2

## Instalación

Instalaremos PyTorch usando conda y sin soporte para CUDA.
Abrir una terminal o cónsola de bash.

1. Activar conda o miniconda con Python 3

        > source ~/miniconda3/bin/activate
        
2. Arear un nuevo environment en conda y activarlo

        > conda create --name env-pytorch
        > conda activate env-pytorch
        
3. Instalar PyTorch. Para ello, ir a la página

    https://pytorch.org/get-started/locally/#start-locally
    
   y elegir:

        PyTorch build      -> Stable
        Your OS            -> Linux
        Package            -> Conda
        Language           -> Python
        Compute platform   -> CPU
    
   Esto generará un comando debajo que deberemos ingresar en el environment de conda recientemente activado

        (env-pytorch)> conda install pytorch torchvision torchaudio cpuonly -c pytorch
        
   Notar que, además de instalar pytorch, también instala torchvision y torchaudio. Estos paquetes incluyen, entre otras cosas, datasets con los cuales experimentar.

4. Para completar el environment, instalar numpy, scipy, scikit-learn, jupyter, matplotlib y pandas
        
        (env-pytorch)> conda install -c anaconda numpy 
        (env-pytorch)> conda install -c conda install -c anaconda scipy 
        (env-pytorch)> conda install -c anaconda scikit-learn 
        (env-pytorch)> conda install -c conda install -c anaconda jupyter
        (env-pytorch)> conda install -c conda-forge matplotlib
        (env-pytorch)> conda install -c anaconda pandas
        
5. Opcionalmente, instalar PyTorchViz usando pip

        (env-pytorch)> conda install -c conda-forge python-graphviz
        (env-pytorch)> pip install torchviz
        
## Referencias

    https://pytorch.org/tutorials/beginner/basics/intro.html
    https://pytorch.org/tutorials/beginner/pytorch_with_examples.html
    https://wiki.pathmind.com/comparison-frameworks-dl4j-tensorflow-pytorch#tensorflow
    https://blog.paperspace.com/ultimate-guide-to-pytorch/

## Testeamos la instalación

Nos basamos en

    https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html

Importamos librerias

In [ ]:
import torch
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt
import numpy as np
import sklearn as skl
from torchviz import make_dot
import torch.optim as optim

## Sobre el uso de tensores de PyTorch

Podemos crear tensores a partir de listas anidadas

In [ ]:
data = [[1, 2],[3, 4]]
x_data = torch.tensor(data)
x_data

... o a partir de numpy arrays

In [ ]:
np_array = np.array(data)
x_np = torch.from_numpy(np_array)
x_np

...o crearlos con valores predefinidos, respetando la forma y tipo de otros tensores prexistentes

In [ ]:
x_ones = torch.ones_like(x_data) # retains the properties of x_data
x_ones

In [ ]:
x_rand = torch.rand_like(x_data, dtype=torch.float) # overrides the datatype of x_data
x_rand

Recordar que shape determina el número y tamaño de las dimensiones del tensor

In [ ]:
shape = (2,3,)
rand_tensor = torch.rand(shape)
ones_tensor = torch.ones(shape)
zeros_tensor = torch.zeros(shape)
print(f"Random Tensor: \n {rand_tensor} \n")
print(f"Ones Tensor: \n {ones_tensor} \n")
print(f"Zeros Tensor: \n {zeros_tensor}")

Como con los arrays de numpy, además de shape y dtype (data type) los tensores de PyTorch tienen el atributo device, el cual especifica en que dispositivo (i.e. en que CPU o GPU) está instanciado el tensor.
Es decir, este atributo es relevante sólo cuando trabajamos con GPUs o en un cluster de computadoras.

In [ ]:
tensor = torch.rand(3,4)
print(f"Shape of tensor: {tensor.shape}")
print(f"Datatype of tensor: {tensor.dtype}")
print(f"Device tensor is stored on: {tensor.device}")

Operaciones con tensores.
Podemos mover o copiar un tensor desde la CPU a la GPU.

In [ ]:
if torch.cuda.is_available():
    tensor = tensor.to('cuda')

En PyTorch el manejo de las componentes de los tensores (escalares, vectoriales, o sub-tensoriales) es equivalente al de numpy.

In [ ]:
tensor = torch.ones(4, 4)
print('First row: ', tensor[0])
print('First column: ', tensor[:, 0])
print('Last column:', tensor[:, -1])
print('Last column:', tensor[..., -1])
tensor[:,1] = 0
print(tensor)

Podemos concatenar tensores (ver también `torch.stack` que trabaja de manera similar)

In [ ]:
t1 = torch.cat([tensor, 2*tensor, 3*tensor], dim=0)
print(t1)

In [ ]:
t1 = torch.cat([tensor, 2*tensor, 3*tensor], dim=1)
print(t1)

Las siguientes, son diferentes formas de multiplicar matricialmente dos tensores.
Aquí `y1`, `y2`, `y3` resultarán con el mismo valor.

In [ ]:
y1 = tensor @ tensor.T

y2 = tensor.matmul(tensor.T)

y3 = torch.rand_like(tensor)
torch.matmul(tensor, tensor.T, out=y3)

De manera similar, estas son diferentes formas de multiplicar punto a punto dos tensores. 
Aquí `z1`, `z2`, `z3` resultarán con el mismo valor.

In [ ]:
z1 = tensor * tensor

z2 = tensor.mul(tensor)

z3 = torch.rand_like(tensor)
torch.mul(tensor, tensor, out=z3)

Podemos, por ejemplo, sumar todas las componentes de un tensor para generar un tensor de una única componente.

In [ ]:
agg = tensor.sum()
print(agg,type(agg)) # Esto es un tensor de PyTorch de una sola componente.

Para acceder al valor numérico en formato Python de dicha componente, usamos el método `.item()`.

In [ ]:
agg_item = agg.item()
print(agg_item,type(agg_item)) # Esto es (algo así como) un número flotante de Python.

In place operations: las funciones miembreo de un tensor que terminan en el caracter `_` corresponden a operaciones que actuan *in place*, i.e. que modifican
las componentes del tensor sin generar una copia.
Por ejemplo, la siguiente expresión suma 5 a cada componente del tensor llamado `tensor`.

In [ ]:
tensor.add_(5)
tensor

Se puede crear una *vista* (view) en formato numpy de un tensor de PyTorch.

In [ ]:
t = torch.ones(5)
n = t.numpy()
print(f"t: {t}")
print(f"n: {n}")

Notar que `n` no es una copia del contenido de `t`, sinó un view.
Entonces, un cambio en el tensor `t` se ve reflejado en `n`.

In [ ]:
t.add_(1)
print(f"t: {t}")
print(f"n: {n}")

## Construyendo y entrenando una red neuronal (1)

### Ejemplo 1: modelo simple, con numpy

Nos interesa aprender un simple modelo de regresión lineal

$$ y = a + bx $$

Comenzamos por generar los datos.
Son $n=100$ puntos, en donde los *features* vienen representados por algunos valores $x_1,x_2,...$ de`x` y los *labels* por correspondientes valores $y_1,y_2,...$ de `y` dados por

$$ y_i = a + bx_i + \epsilon_i $$

donde $a=1$, $b=2$ y $\epsilon_i \sim \mathcal{N}(0,1)$ es un número aleatorio generado de una distribución gaussia de media 0 y varianza 1.

In [ ]:
# Data Generation
np.random.seed(42)
x = np.random.rand(100, 1) # features
y = 1 + 2 * x + .1 * np.random.randn(100, 1) # labels

Luego de aleatorizar el orden de los puntos, dividimos los datos en un conjunto de entrenamiento (de tamaño 80) y uno de validación (de tamaño 20).

In [ ]:
# Shuffles the indices
idx = np.arange(100)
np.random.shuffle(idx)

# Uses first 80 random indices for train
train_idx = idx[:80]
# Uses the remaining indices for validation
val_idx = idx[80:]

# Generates train and validation sets
x_train, y_train = x[train_idx], y[train_idx]
x_val, y_val = x[val_idx], y[val_idx]

In [ ]:
fig,axes=plt.subplots(1,2)
fig.set_size_inches(5.0,3.0)
ax = axes[0]
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_title("training")
ax.scatter(x_train,y_train)
ax = axes[1]
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_title("validation")
ax.scatter(val_idx,val_idx,c='r')
fig.tight_layout()
plt.show()

El objetivo último es aprender los valores de los parámetros $a$ y $b$ usados para generar los datos.
Para ello consideramos la función de pérdida definida por el Error Cuadrático Medio entre las predicciones $\hat{y}_i:=y(x_i)$ y los valores labels $y_i$

\begin{eqnarray}
L(x,y;a,b) &=& \frac{1}{n}\sum_{i=1}^n (y_i-y(x_i))^2 \\
&=& \frac{1}{n}\sum_{i=1}^n (y_i-a-bx_i)^2 \\
\end{eqnarray}

Usaremos el método del descenso por el gradiente

$$ \frac{\partial L^{(t)}}{\partial a}(x,y;a^{(t)},b^{(t)}) = -\frac{2}{n}\sum_{i=1}^n(y_i-y_i(x_i)) $$

$$ \frac{\partial L^{(t)}}{\partial b}(x,y;a^{(t)},b^{(t)}) = -\frac{2}{n}\sum_{i=1}^nx_i(y_i-y_i(x_i)) $$

arrancando con valores iniciales de los parámetros $a=a^{(0)}$ y $b=b^{(0)}$ elegidos al azar de alguna distribución de probabilidades, para luego iterar sobre la época de entrenamiento $t$ según

$$ a^{(t+1)} = a^{(t)} - \eta \frac{\partial L^{(t)}}{\partial a}(x,y;a^{(t)},b^{(t)}) $$

$$ b^{(t+1)} = b^{(t)} - \eta \frac{\partial L^{(t)}}{\partial b}(x,y;a^{(t)},b^{(t)}) $$

donde el $\eta$ tal que $0<\eta\ll 1$, es la tasa de aprendizaje.

In [ ]:
# Initializes parameters "a" and "b" randomly
np.random.seed(42)
a = np.random.randn(1)
b = np.random.randn(1)

print(f"Iniciales a={a},b={b}")

# Sets learning rate
lr = 1e-1

# Defines number of epochs
n_epochs = 1000

for epoch in range(n_epochs):
    # Computes our model's predicted output
    yhat = a + b * x_train
    
    # How wrong is our model? That's the error! 
    error = (y_train - yhat)
    # It is a regression, so it computes mean squared error (MSE)
    loss = (error ** 2).mean()
    
    # Computes gradients for both "a" and "b" parameters
    a_grad = -2 * error.mean()
    b_grad = -2 * (x_train * error).mean()
    
    # Updates parameters using gradients and the learning rate
    a = a - lr * a_grad
    b = b - lr * b_grad

print(f"Ajustados a={a},b={b}")

# Sanity Check: do we get the same results as our gradient descent?
from sklearn.linear_model import LinearRegression
linr = LinearRegression()
linr.fit(x_train, y_train)
print(f"Check     a={linr.intercept_},b={linr.coef_[0]}")

### Ejemplo 2: idem, pero con PyTorch.
Convenientemente, PyTorch provee de `torch.autograd`, un motor de diferenciación automática para calcular gradientes.

Para ello, en este ejemplo tenemos que decirle a torch que `a` y `b` son tensores que requieren gradientes.

In [ ]:
# Our data was in Numpy arrays, but we need to transform them into PyTorch's Tensors
x_train_tensor = torch.from_numpy(x_train).float()
y_train_tensor = torch.from_numpy(y_train).float()

# Initializes parameters "a" and "b" randomly, ALMOST as we did in Numpy
# since we want to apply gradient descent on these parameters, we need
# to set REQUIRES_GRAD = TRUE
a = torch.randn(1, requires_grad=True, dtype=torch.float)
b = torch.randn(1, requires_grad=True, dtype=torch.float)
print(f"Iniciales a={a.item()},b={b.item()}")

Entrenemos el modelo con un optimizador personalizado

In [ ]:
lr = 1e-1
n_epochs = 1000

torch.manual_seed(42)
a = torch.randn(1, requires_grad=True, dtype=torch.float) #, device=device)
b = torch.randn(1, requires_grad=True, dtype=torch.float) #, device=device)
print(f"Iniciales a={a.item()},b={b.item()}")

for epoch in range(n_epochs):
    yhat = a + b * x_train_tensor
    error = y_train_tensor - yhat
    loss = (error ** 2).mean()

    # No more manual computation of gradients! 
    # a_grad = -2 * error.mean()
    # b_grad = -2 * (x_tensor * error).mean()
    
    # We just tell PyTorch to work its way BACKWARDS from the specified loss!
    loss.backward()
    # Let's check the computed gradients...
    print(f"epoch={epoch} a.grad={a.grad},b.grad={b.grad}")    
    
    # We need to use NO_GRAD to keep the update out of the gradient computation
    # Why is that? It boils down to the DYNAMIC GRAPH that PyTorch uses...
    with torch.no_grad():
        a -= lr * a.grad
        b -= lr * b.grad
    
    # PyTorch is "clingy" to its computed gradients, we need to tell it to let it go...
    a.grad.zero_()
    b.grad.zero_()
    
print(f"Ajustados a={a.item()},b={b.item()}")

Notar que hemos usado `with torch.no_grad()` al momento de actualizar los valores de los parámetros `a` y `b`.

Esto es así, porque de otra manera deberíamos cambiar el valor de `a.grad` y `b.grad` al actualizar los valores de `a` y `b`.
Por alguna razón, a PyTorch no le gusta esto (quizás para evitar evalauciones ciruclares) y, por ende, si querémos actualizar los valores de `a` y `b`, tenemos que decirle primero a PyTorch que desactive la actualización automática de los gradientes.

PyTorch calcula los gradientes de forma acumulativa.
Por ello, cada vez que usamos los gradientes para actualizar los parámetros, luego debemos resetear a 0 los gradientes usando, por ejemplo, el método `.zero()`.

En PyTorch los gradientes son actualizados tal como es especificado por el grafo computacional determinado por el modelo que uno define

![](comp-graph-ej1.png "")

In [ ]:
# A estos gráficos de arriba se los puede generar con torchviz
#make_dot(yhat)
#make_dot(error)
#make_dot(loss)

### Ejemplo 3: entrenando modelos con el optimizador proveido por PyTorch.

Además del cálculo automático de gradientes, PyTorch provee de un optimizador que se encarga de la actualización automática de parámetros.

In [ ]:
torch.manual_seed(42)
a = torch.randn(1, requires_grad=True, dtype=torch.float) #, device=device)
b = torch.randn(1, requires_grad=True, dtype=torch.float) #, device=device)
print(f"Iniciales a={a.item()},b={b.item()}")

lr = 1e-1
n_epochs = 1000

# Defines a SGD optimizer to update the parameters
optimizer = optim.SGD([a, b], lr=lr)

for epoch in range(n_epochs):
    yhat = a + b * x_train_tensor
    error = y_train_tensor - yhat
    loss = (error ** 2).mean()

    loss.backward()    
    
    # No more manual update!
    # with torch.no_grad():
    #     a -= lr * a.grad
    #     b -= lr * b.grad
    optimizer.step()
    
    # No more telling PyTorch to let gradients go!
    # a.grad.zero_()
    # b.grad.zero_()
    optimizer.zero_grad()
    
print(f"Ajustados a={a.item()},b={b.item()}")

### Ejemplo 4:

https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html

Primero chequeamos si hay disponibles placas GPU con CUDA, o no.

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Usando el dispositivo {}'.format(device))

Luego, definimos nuestro modelo como una clase derivada de la clase `nn.Model`.
Inicializamos la red neuronal usando el metodo`.__init__()`.
Toda subclase de `nn.Module` implementa el como operar sobre datos de entrada (input) para generar una salida (output) a travéz del método `.forward()`.

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        # Primero convertimos los datos de entrada a un gran vector.
        self.flatten = nn.Flatten()
        # Luego generamos una secuencia de transformaciones que implementan las capas de la red.
        # Algunas capas consisten en transformaciones lineales, que incluyen parámetros.
        # Otras consisten en funciones de activación no lineales, que no incluyen parámetros.
        # Los tamaños de las capas se especifican a travez de los tamaños de las transformaciones lineales.
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        # Este método es usado internamente para transformar inputs en outputs en las fases forward.
        # Al igual que antes, aplanamos los datos de entrada en un gran vector.
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

Luego creamos una un objeto (o instancia) de la clase `NeuralNetwork`, y lo movemos al dispositivo que usaremos.
Además, imprimimos su estructura para ver si todo salió como esperábamos.

In [ ]:
model = NeuralNetwork().to(device)
print(model)

Para usar el modelo, le pasamos datos de entrada.
Esto ejecuta el método `.forward()`, junto con algunas otras operaciones de fondo.
Ojo! No hay que llamar `model.forward()` directamente!

Tras llamar el modelo sobre algunos inputs, este retorna un tensor 10-dimensional con algunos valores crudos o *raw* predecidos para cada clase.
Obtenemos de estos probabilidades de predicción por medio de pasarlos a travéz de una instancia del modulo `nn.Softmax`.

In [ ]:
# Generamos datos de entrada truchos (una imagen de píxeles aleatorios).
X = torch.rand(1, 28, 28, device=device)

# Se los damos de comer al modelo via el método .__call__() del mismo.
# Notar que este modelo no está entrenado, por lo que el output será cualquier cosa.
logits = model(X)

# Imprimimos la salida cruda (raw) generado por el modelo
print(f"Salida del modelo = {logits}")
print("")

# Convertimos estos valores crudos a probabilidades y las imprimimos
pred_probab = nn.Softmax(dim=1)(logits)
print(f"Probabilidades predecidas = {pred_probab}")
print("")

# Elegimos la categoría asociada a la mayor probabilidad.
y_pred = pred_probab.argmax(1)
print(f"Categoría predecida: {y_pred}")

#### Desglocemos el modelo

# Desglocemos lo que ocurre en las capas del modelo de FashionMNIST.
# Para ellos, tomaremos una muestra minilote de 3 imagenes de tamaño 28x28,
# y veremos que ocurre si se lo pasamos a la red.

input_image = torch.rand(3,28,28)
print(input_image.size())

# Inicializamos la capa nn.Flatten para convertir cada imagen 2D de 28x28 pixeles
# a un array de 28*x28=748 valores contiguos (la dimensión de minibatch (dim=0) 
# se mantiene).

flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

# Una capa lineal es un modulo que aplica una transformación lineal a la entrada,
# usando los pesos (weights) y sesgos (biases) almacenados.

layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

# Las unidades no lineales de activación sirven para lograr mapeos complejos entre
# inputs y outputs.
# Estas se aplican después de transformaciones lineales.
# En este ejemplo, usamos nn.ReLUs entre las capas lineales.

print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

# nn.Sequential is contenedor ordenado de módulos.
# Los datos de entrada son pasados a travéz de los módulos en el mismo orden
# en que fueron definidos.
# Uno puede usar contenedores secuenciales para rápidamente crear una red como 
# seq_modules.

seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

# La ultima capa de la red neuronal retorna procesando los datos con la función logit,
# pasándolos por el módulo nn.Softmax
# La función logit rescalea valores en [-infty,infty] a valores en [0,1]
# para que estos valores representen probabilidades de predicción para cada clase.
# El parámetro indica la dimensión sobre la cual los valores deben sumar 1.

softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

# Los parámetros del modelo

# Varias de las capas de una red neuronal están parametrizadas, i.e. poseen 
# pesos (weights) y tendencias (biases) que son optimizados durante el entrenamiento.
# Al subclasear nn.Module, automaticamente tenemos en cuenta todos lso campos
# definidos dentro del objeto *model*, y hace accessibles todos los parametros
# allí definidos por medio de los métodos parameters() o named_parameters().
# En este ejemplo, iteramos sobre todos los parámetros e imprimimos sus tamaños
# y un preview de sus valores.

print("Model structure: ", model, "\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")